In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
# import kagglehub
# rounakbanik_the_movies_dataset_path = kagglehub.dataset_download('rounakbanik/the-movies-dataset')

# print('Data source import complete.')


# Data load, clean and format

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
%pip install langid
import langid
import time
import ast
import timeit

%pip install transformers torch
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [ ]:
import configparser
config = configparser.ConfigParser()
config_file = "config.ini"
config.read(config_file)

In [ ]:

movies_file = config["FILES"]["MOVIES_FILE"]
ratings_file = config["FILES"]["RATINGS_FILE"]
credits_file = config["FILES"]["CREDITS_FILE"]

movies_df = pd.read_csv(movies_file, low_memory=False)
ratings_df = pd.read_csv(ratings_file)
credits_df = pd.read_csv(credits_file)

OUTPUT_FILE = "output/output.txt"

In [ ]:
movies_df.shape

In [ ]:
ratings_df.shape

In [ ]:
credits_df.shape

In [ ]:
movies_df.dtypes

In [ ]:
ratings_df.dtypes

In [ ]:
credits_df.dtypes

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
credits_df.head()

In [ ]:
movies_df["id"].nunique()

In [ ]:
ratings_df["movieId"].nunique()

In [ ]:
credits_df["id"].nunique()

## Dataframes Cleaning

In [ ]:
movies_df_columns = ["id", "title", "genres", "release_date", "overview", "production_countries", "spoken_languages", "budget", "revenue"]
ratings_df_columns = ["movieId", "rating", "timestamp"]
credits_df_columns = ["id", "cast"]

In [ ]:
# Discard unuseful columns
movies_df_cleaned = movies_df.dropna(subset=movies_df_columns)[movies_df_columns].copy()
ratings_df_cleaned = ratings_df.dropna(subset=ratings_df_columns)[ratings_df_columns].copy()
credits_df_cleaned = credits_df.dropna(subset=credits_df_columns)[credits_df_columns].copy()

In [ ]:
# Dates Formatting
movies_df_cleaned['release_date'] = pd.to_datetime(movies_df_cleaned['release_date'], format='%Y-%m-%d', errors='coerce')
ratings_df_cleaned['timestamp'] = pd.to_datetime(ratings_df_cleaned['timestamp'], unit='s')

In [ ]:
# Numbers Formatting
movies_df_cleaned['budget'] = pd.to_numeric(movies_df_cleaned['budget'], errors='coerce')
movies_df_cleaned['revenue'] = pd.to_numeric(movies_df_cleaned['revenue'], errors='coerce')

In [ ]:
# Replace json fields with string arrays
def dictionary_to_list(dictionary_str):
    try:
        dictionary_list = ast.literal_eval(dictionary_str)
        return [data['name'] for data in dictionary_list]
    except (ValueError, SyntaxError):
        return []

movies_df_cleaned['genres'] = movies_df_cleaned['genres'].apply(dictionary_to_list)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].apply(dictionary_to_list)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].apply(dictionary_to_list)
credits_df_cleaned['cast'] = credits_df_cleaned['cast'].apply(dictionary_to_list)

In [ ]:
movies_df_cleaned['genres'] = movies_df_cleaned['genres'].astype(str)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].astype(str)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].astype(str)

In [ ]:
movies_df_cleaned.head()

In [ ]:
ratings_df_cleaned.head()

In [ ]:
credits_df_cleaned.head()

In [ ]:
movies_df_cleaned.dtypes

In [ ]:
ratings_df_cleaned.dtypes

In [ ]:
credits_df_cleaned.dtypes

# Queries a Resolver

- Q1: Peliculas y sus géneros de los años 2000 con producción Argentina y Española.
- Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.
- Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.
- Q4: Top 10 de actores con mayor participación en películas de producción Argentina con
fecha de estreno posterior al 2000
- Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo, para películas de habla inglesa con producción americana, estrenadas a partir del año 2000

### Q1:  Peliculas y sus géneros de los años 00' con producción Argentina y Española Q2:

In [ ]:
movies_argentina_españa_00s_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['production_countries'].str.contains('Spain', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000) &
    (movies_df_cleaned['release_date'].dt.year < 2010)
]

In [ ]:
movies_argentina_españa_00s_df.shape


In [ ]:
movies_argentina_españa_00s_df[["title", "genres"]]

with open(OUTPUT_FILE, "w") as f:
    f.write("query 1\n")
    for index, row in movies_argentina_españa_00s_df.iterrows():
        title = row["title"]
        genres = row["genres"]
        f.write(f"{title},{genres}\n")
    f.write("\n")

In [ ]:
movies_argentina_españa_00s_df[["title", "genres"]]

### Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.

In [ ]:
solo_country_df = movies_df_cleaned[movies_df_cleaned['production_countries'].apply(lambda x: len(eval(x)) == 1)]

In [ ]:
solo_country_df.loc[:, 'country'] = solo_country_df['production_countries'].apply(lambda x: eval(x)[0])

In [ ]:
investment_by_country = solo_country_df.groupby('country')['budget'].sum().sort_values(ascending=False)

In [ ]:
top_5_countries = investment_by_country.head(5).reset_index()

print(top_5_countries)

In [ ]:
with open(OUTPUT_FILE, "a") as f:
    f.write("query 2\n")
    for index, row in top_5_countries.iterrows():
        country = row["country"]
        budget = row["budget"]
        f.write(f"{country},{budget}\n")
    f.write("\n")

### Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.

In [ ]:
movies_argentina_post_2000_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000)
]

In [ ]:
movies_argentina_post_2000_df = movies_argentina_post_2000_df.astype({'id': int})

In [ ]:
ranking_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(ratings_df_cleaned,
                                                                                left_on="id",
                                                                                right_on="movieId")
mean_ranking_arg_post_2000_df = ranking_arg_post_2000_df.groupby(["id", "title"])['rating'].mean().reset_index()

In [ ]:
#Max
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmax()]




In [ ]:
#Min
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmin()]

In [ ]:
title_max = mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmax()]["title"]
rating_max = mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmax()]["rating"]

title_min = mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmin()]["title"]
rating_min = mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmin()]["rating"]

with open(OUTPUT_FILE, "a") as f:
    f.write("query 3\n")
    f.write(f"max, {title_max},{rating_max}\n")
    f.write(f"min, {title_min},{rating_min}\n")
    f.write("\n")

### Q4: Top 10 de actores con mayor participación en películas de producción Argentina posterior al 2000

In [ ]:
cast_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(credits_df_cleaned,
                                                                                on="id")
cast_and_movie_arg_post_2000_df = cast_arg_post_2000_df.set_index("id")["cast"].apply(pd.Series).stack().reset_index("id", name="name")
cast_per_movie_quantities = cast_and_movie_arg_post_2000_df.groupby(["name"]).count().reset_index().rename(columns={"id":"count"})
cast_per_movie_quantities.nlargest(10, 'count')

with open(OUTPUT_FILE, "a") as f:
    f.write("query 4\n")
    for index, row in cast_per_movie_quantities.nlargest(10, 'count').iterrows():
        name = row["name"]
        count = row["count"]
        f.write(f"{name},{count}\n")
    f.write("\n")

### Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo

In [ ]:
q5_input_df = movies_df_cleaned.copy()

In [ ]:
# Nos quedamos con rows válidas
q5_input_df = q5_input_df.loc[q5_input_df['budget'] != 0]
q5_input_df = q5_input_df.loc[q5_input_df['revenue'] != 0]

In [ ]:
q5_input_df.shape

In [ ]:
# Cargar modelo preentrenado para análisis de sentimiento
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

In [ ]:
start_time = time.time()
q5_input_df['sentiment'] = q5_input_df['overview'].fillna('').apply(lambda x: sentiment_analyzer(x)[0]['label'])
elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

In [ ]:
q5_input_df["rate_revenue_budget"] = q5_input_df["revenue"] / q5_input_df["budget"]

In [ ]:
q5_input_df.sample(10)

In [ ]:
average_rate_by_sentiment = q5_input_df.groupby("sentiment")["rate_revenue_budget"].mean()
print(average_rate_by_sentiment)